In [1]:
// TypeScript Jupyter extension
import * as tslab from "tslab";

// CSC 600 Libraries
import { drawTree, requireCytoscape, requireCarbon} from "./lib/draw";

requireCarbon();
requireCytoscape();

# Algebraic Data Types: Recursion with Data Types

## Where Were We?

1. **Language primitives** (i.e., building blocks of languages)
    * Last time: recursive thinking with functions
    * This time: recursive thinking with data, i.e., **algebraic data types** (ADT).
2. Language paradigms (i.e., combinations of language primitives)
3. Building a language (i.e., designing your own language)

## Goal

1. Introduce **(algebraic) data-types** (**ADT**) using **trees** which are an example of a recursive data type.
2. Compare with class-based definition of tree.

## Outline

- Why ADTs?
- Trees

## Why ADTs?

Example:  [https://www.paulrand.design/work/Book-Covers.html](https://www.paulrand.design/work/Book-Covers.html)

Here's a question for you first.

```








Imagine you're a graphic designer.
You want to layout a book cover with:
1. Text
2. Drawings
in an aesthetically pleasing way.


How would you tackle the problem of laying out such a book cover?









```

In [2]:
tslab.display.html(`
<div class="bx--tile">
    BOX 1: Imagine I'm a page
</div>
`)

BOX 1: Imagine I'm a page

In [3]:
tslab.display.html(`
   <div class="bx--grid"> BOX 1: Imagine I'm a page
      <div class="bx--row">
        <div class="bx--col">
            <div class="bx--tile">
                BOX 2: Left side of page
            </div>
        </div>
        <div class="bx--col">
            <div class="bx--tile">
                BOX 3: Right side of page
            </div>
        </div>
    </div>
`)

BOX 1: Imagine I'm a page
 
 
 
 BOX 2: Left side of page
 
 
 
 
 BOX 3: Right side of page

In [4]:
tslab.display.html(`
<div class="bx--grid"> BOX 1: Imagine I'm a page
    <div class="bx--row">
        <div class="bx--col">
            <div class="bx--tile">
                BOX2: Left side of page
                <div class="bx--tile">
                    BOX5: A sub-page within the left page
                </div>
            </div>
        </div>
        <div class="bx--col">
            <div class="bx--tile">
                BOX3: Right side of page
                
                <div class="bx--tile">
                    BOX4: A sub-page within the right page
                </div>
            </div>
        </div>
    </div>
</div>
`)

BOX 1: Imagine I'm a page
 
 
 
 BOX2: Left side of page
 
 BOX5: A sub-page within the left page
 
 
 
 
 
 BOX3: Right side of page
 
 
 BOX4: A sub-page within the right page

### Notice a Pattern?

- To solve the graphic design problem, we're "nesting" boxes within boxes.
- This is starting to look like recursion.
    - Maybe I don't know how to layout the full page.
    - I'll solve a simpler version of the same problem by splitting the page into different smaller pages.
    - Now let me solve the layout problem for the smaller pages.
    - And put together the results for the original page.
- Instead of writing a recursive function, we'll create a recursive data structure.

## Trees

- We'll introduces trees now.
- Trees are the canoncial recursive data structure.

In [5]:
import * as tree from "./lib/tree";

drawTree(tree.t4)

### Refresher: Trees with Classes

In [6]:
class TreeCls<T> {
    private contents: T | undefined;        // e.g., number or strings
    private left: TreeCls<T> | undefined;   // left child or undefined
    private right: TreeCls<T> | undefined;  // right child or undefined
    
    constructor(contents: T|undefined, left: TreeCls<T>|undefined, right: TreeCls<T>|undefined) {
        this.contents = contents;
        this.left = left;
        this.right = right;
    }
    
    toADT() { // Ignore for now 
        if (this.contents === undefined && this.left === undefined && this.right === undefined) {
            return {
                tag: "LEAF"
            };
        } else {
            return {
                tag: "NODE",
                contents: this.contents,
                left: this.left !== undefined ? this.left.toADT() : { tag: "LEAF" },
                right: this.right !== undefined ? this.right.toADT() : { tag: "LEAF" }
            };
        }
    }
}

In [7]:
function newLeaf<T>(): TreeCls<T> {
    return new TreeCls(undefined, undefined, undefined);
}

function newNode<T>(x: T, left: TreeCls<T>, right: TreeCls<T>): TreeCls<T> {
    return new TreeCls(x, left, right);
}

function newLeafNode<T>(x: T): TreeCls<T> {
    return new TreeCls(x, undefined, undefined);
}

#### Leaf

In [8]:
drawTree(newLeaf().toADT());

#### Leaf Node

In [9]:
drawTree(newLeafNode(1).toADT());

#### Node

In [10]:
drawTree(newNode(3, newLeafNode(1), newLeafNode(2)).toADT());

In [11]:
drawTree(newNode(3, newLeafNode(1), newLeaf()).toADT());

#### Tree

In [12]:
const t1 = newLeafNode(1);
const t2 = newNode(2, t1, newLeaf());
const t3 = newNode(3, t1, newLeafNode(2));
const t4 = newNode(4, t3, t2);
drawTree(t4.toADT());

### Trees with Algebraic Data-Types (ADTs)

- Now we'll do the same thing again, this time without classes.
- Instead we'll use sum types and record types.
- In a few lectures, we'll see a comparison of OOP (classes) vs. functional (ADTs).

In [13]:
// Tree data-type
type Tree<T> =
    {                      // record type
        tag: "LEAF"        // literal type
    }
|   {                      // sum type and record type
        tag: "NODE",
        contents: T,       // notice no undefined
        left: Tree<T>,
        right: Tree<T>
    };

In [14]:
function mkLeaf<T>(): Tree<T> { // compare with newLeaf
    // Constructor function for a node with no contents.
    return {
        tag: "LEAF"
    };
}

function mkNode<T>(x: T, left: Tree<T>, right: Tree<T>): Tree<T> { // compare with newNode
    // Constructor function for a node with contents, a left sub-tree, and a right-subtree.
    return {
        tag: "NODE",
        contents: x,
        left: left,
        right: right
    };
}

function mkLeafNode<T>(x: T): Tree<T> { // compare with newLeafNode
    return mkNode(x, mkLeaf(), mkLeaf());
}

In [15]:
const x = {
    tag: "NODE",
    contents: 1,
    left: { tag: "LEAF" },
    right: { tag: "LEAF" }
}
x

{
  tag: 'NODE',
  contents: 1,
  left: { tag: 'LEAF' },
  right: { tag: 'LEAF' }
}


#### Book cover design with trees

In [16]:
const t5 = mkLeafNode("Box 5: left Bot");
const t0 = mkNode("Box2: Left", t5, mkLeaf());
const t1 = mkLeafNode("Box 4: Right Bot");
const t2 = mkNode("Box 3: Right", t1, mkLeaf());
const t3 = mkNode("Box 1", t0, t2);

In [17]:
drawTree(t3)

In [18]:
drawTree(t1)

In [19]:
drawTree(t2)

In [20]:
drawTree(t3)

In [21]:
t1

{
  tag: 'NODE',
  contents: 'Box 4: Right Bot',
  left: { tag: 'LEAF' },
  right: { tag: 'LEAF' }
}


In [22]:
t2 // Notice nesting of t1 in t2's left child

{
  tag: 'NODE',
  contents: 'Box 3: Right',
  left: {
    tag: 'NODE',
    contents: 'Box 4: Right Bot',
    left: { tag: 'LEAF' },
    right: { tag: 'LEAF' }
  },
  right: { tag: 'LEAF' }
}


In [23]:
t3 // Notice nesting of t2 in t3's right child

{
  tag: 'NODE',
  contents: 'Box 1',
  left: {
    tag: 'NODE',
    contents: 'Box2: Left',
    left: {
      tag: 'NODE',
      contents: 'Box 5: left Bot',
      left: [Object],
      right: [Object]
    },
    right: { tag: 'LEAF' }
  },
  right: {
    tag: 'NODE',
    contents: 'Box 3: Right',
    left: {
      tag: 'NODE',
      contents: 'Box 4: Right Bot',
      left: [Object],
      right: [Object]
    },
    right: { tag: 'LEAF' }
  }
}


#### Recursive Functions on Trees

In [24]:
drawTree(t2)

#### Computing the height of a tree

```ts

height(mkNode("Box 3: Right"), mkLeafNode("Box 4: Right Bot"), mkLeaf()))
    = 1 + max(height(mkLeafNode("Box 4: Right Bot")), height(mkLeaf())))
    = 1 + max(1 + max(height(mkLeaf()), height(mkLeaf())), 0)
    = 1 + max(1 + max(0, 0), 0)
    = 1 + 1
    = 2
```

In [25]:
function height<T>(t: Tree<T>): number {
    switch (t.tag) {
        case "LEAF": {   // Base case: leaf node
            return 0;
        }
        case "NODE": {   // Recursive (or inductive) case: node with left and right child
            return 1 + Math.max(height(t.left), height(t.right));
        }
    }
}

height(t2)

2


#### Converting a tree to a string

```ts

trToStr(mkNode("Box 3: Right"), mkLeafNode("Box 4: Right Bot"), mkLeaf()))
    = `(Box 3: Right ${trToStr(mkLeafNode("Box 4: Right Bot"))} ${trToStr(mkLeaf())})`
    = `(Box 3: Right (Box 4: Right Bot ${trToStr(mkLeaf())} ${trToStr(mkLeaf())}) ())`
    = `(Box 3: Right (Box 4: Right Bot () ()) ())`
```

In [26]:
function trToStr<T>(t: Tree<T>): string {
    switch (t.tag) {
        case "LEAF": {  // Base case: by definition "()"
            return "()";  
        }
        case "NODE": {  // Recursive case: by definition "()"
            // Assume: treeToString(t.left) gives me the answer to the left child 
            // Assume: treeToString(t.right) gives me the answer to the right child
            // Answer (contents left right)
            return `(${t.contents.toString()} ${trToStr(t.left)} ${trToStr(t.right)})`;
        }
            
    }
}

In [27]:
trToStr(t2)

(Box 3: Right (Box 4: Right Bot () ()) ())


#### Mapping a tree

In [28]:
function mapTree<T, U>(f: (arg: T) => U, t: Tree<T>): Tree<U> {
    switch (t.tag) {
        case "LEAF": {
            return mkLeaf();
        }
        case "NODE": {
            return mkNode(f(t.contents), mapTree(f, t.left), mapTree(f, t.right));
        }
            
    }
}

In [29]:
drawTree(t2)

In [30]:
drawTree(mapTree((arg: string) => arg + "!!!!", t2))

#### What does the class version look like?

- Notice how we have to modify the entire class.
- Lookahead to OOP vs. functional: easier to add functions in functional style.
- What's easier to add for OOP?

In [31]:
class TreeCls<T> {
    private contents: T | undefined;
    private left: TreeCls<T> | undefined;
    private right: TreeCls<T> | undefined;
    
    constructor(contents: T|undefined, left: TreeCls<T>|undefined, right: TreeCls<T>|undefined) {
        this.contents = contents;
        this.left = left;
        this.right = right;
    }
    
    map<U>(f: (arg: T) => U) { // Compare and contrast!
        if (this.contents === undefined && this.left === undefined && this.right === undefined) {
            return {
                tag: "LEAF"
            };
        } else {
            return {
                tag: "NODE",
                contents: f(this.contents),
                left: this.left !== undefined ? this.left.map(f) : { tag: "LEAF" },
                right: this.right !== undefined ? this.right.map(f) : { tag: "LEAF" }
            };
        }
    }
    
    toADT() { // Ignore for now 
        if (this.contents === undefined && this.left === undefined && this.right === undefined) {
            return {
                tag: "LEAF"
            };
        } else {
            return {
                tag: "NODE",
                contents: this.contents,
                left: this.left !== undefined ? this.left.toADT() : { tag: "LEAF" },
                right: this.right !== undefined ? this.right.toADT() : { tag: "LEAF" }
            };
        }
    }
}

#### What does the iterative solutions look like?

- Are you sure you don't like recursion anymore?

In [32]:
function mapIterTree<T, U>(f: (arg: T) => U, t: Tree<T>): Tree<U> {
    let callStack: [string, Tree<T> | U][] = [];
    let ans: Tree<U>[] = [];
    callStack.push(["CALL", t]);
    
    while (callStack.length > 0) {
        const [mode, trOrU] = callStack.pop();
        if (mode === "CALL") {
            const tr = trOrU as Tree<T>;
            switch (tr.tag) {
                case "LEAF": {
                    ans.push(mkLeaf());
                    break;
                }
                case "NODE": {
                    callStack.push(["RET", f(tr.contents)]);
                    callStack.push(["CALL", tr.right]);
                    callStack.push(["CALL", tr.left]);
                    break;
                }
            }
        } else if (mode === "RET") {
            const u = trOrU as U;
            const right = ans.pop();
            const left = ans.pop();
            ans.push(mkNode(u, left, right));
        } else {
            throw Error("Shouldn't happen ...");
        }
    }
    
    return ans.pop();
}

In [33]:
drawTree(mapIterTree((arg: string) => arg + "!", t2))

### Summary

- You can do the **same** computations with both iteration and recursion.
- Recursion uses more **stack frames** (the traceCallStack picture) than iteration.
- For some functions, iteration requires you to simulate the stack on the heap (e.g., `mapIterTree`)

In [34]:
tslab.display.html(`
<div class="bx--grid"> Program 1: I'm a top-level program
    <div class="bx--row">
        <div class="bx--col">
            <div class="bx--tile">
                BOX2: Sub-program 2
            </div>
        </div>
        <div class="bx--col">
            <div class="bx--tile">
                BOX3: Sub-program 3
                
                <div class="bx--tile">
                    BOX4: A sub-sub-program
                </div>
            </div>
        </div>
    </div>
</div>
`)

Program 1: I'm a top-level program
 
 
 
 BOX2: Sub-program 2
 
 
 
 
 BOX3: Sub-program 3
 
 
 BOX4: A sub-sub-program

## Summary

- We motivated ADTs + recursion as a natural way of solving a problem such as page layout.
- We looked at an example of an ADT called a Tree.
- Trees are surprisingly powerful. And in fact every TypeScript object underneath looks a lot like a Tree.
- We'll continue to see recursive-style thinking throughout the class.